In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.16.0 to work with agogemls


In [2]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'titanic-training'
os.makedirs(training_folder, exist_ok=True)

In [16]:
%%writefile $training_folder/titanic_training.py

from azureml.core import Run
from azureml.core import Dataset
import pandas
import argparse
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.05, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# load the titanic dataset
print("Loading Data...")
train_df = run.input_datasets['Titanic'].to_pandas_dataframe()

#Data cleansing & Feature engineering
train_df = train_df.drop(['Ticket', 'Cabin', 'PassengerId'], axis=1)

train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
train_df['Title'] = train_df['Title'].fillna(0).astype(int)
train_df = train_df.drop(['Name'], axis=1)

train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
train_df['IsAlone'] = 0
train_df.loc[train_df['FamilySize'] == 1, 'IsAlone'] = 1
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)

train_df['Sex'] = train_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

freq_port = train_df.Embarked.dropna().mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)
train_df['Embarked'] = train_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

guess_ages = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        guess_df = train_df[(train_df['Sex'] == i) & (train_df['Pclass'] == j + 1)]['Age'].dropna()
        age_guess = guess_df.median()
        # Convert random age float to nearest .5 age
        guess_ages[i, j] = int(age_guess / 0.5 + 0.5) * 0.5

for i in range(0, 2):
    for j in range(0, 3):
            train_df.loc[(train_df.Age.isnull()) & (train_df.Sex == i) & (train_df.Pclass == j + 1),
                         'Age'] = guess_ages[i, j]

train_df['Age'] = train_df['Age'].astype(int)

train_df.loc[ train_df['Fare'] <= 7.91, 'Fare'] = 0
train_df.loc[(train_df['Fare'] > 7.91) & (train_df['Fare'] <= 14.454), 'Fare'] = 1
train_df.loc[(train_df['Fare'] > 14.454) & (train_df['Fare'] <= 31), 'Fare']   = 2
train_df.loc[ train_df['Fare'] > 31, 'Fare'] = 3
train_df['Fare'] = train_df['Fare'].astype(int)

# Separate features and labels
X = train_df.drop("Survived", axis=1)
y = train_df["Survived"]

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/titanic_model.pkl')

# Complete the run
run.complete()

Overwriting titanic-training/titanic_training.py


In [18]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.core import Experiment

# Get the training dataset
titanic_ds = ws.datasets.get("Titanic")

# Set the script parameters
script_params = {
    '--regularization': 0.4
}

# Create an estimator
estimator = SKLearn(source_directory=training_folder,
                    entry_script='titanic_training.py',
                    script_params=script_params,
                    compute_target='local',
                    inputs=[titanic_ds.as_named_input('Titanic')], # Pass the Dataset object as an input...
                    pip_packages=['azureml-dataprep[pandas]'] # ...so you need the dataprep package
                    )

# Create an experiment
experiment_name = 'Titanic-experiment-2'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'Titanic-experiment-2_1603282648_f8f216e2',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-10-21T12:17:31.741821Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '2b6ec22e-1a36-4b8c-86e5-3ed3af51d108'},
 'inputDatasets': [{'dataset': {'id': '6256dd80-b31c-4d77-8316-d962ab3f41c1'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'Titanic', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'titanic_training.py',
  'command': [],
  'useAbsolutePath': False,
  'arguments': ['--regularization', '0.4'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'Titanic': {'dataLocation': {'dataset': {'id': '6256dd80-b31c-4d77-8316-d962ab3f41c1',
      'name': 'Titanic',
      'version': '1'},
     'dataPath': None},
    'mechanism': 'Direct',
    'environmentVariableName': 'Titanic',
    'pathOnCompute': None,


In [19]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.4
Accuracy 0.7947761194029851
AUC 0.822945283882784


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/9_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/dataprep/engine_spans_l_0b0ba655-6f7c-4edf-9a6a-ccb462e385ae.jsonl
logs/azureml/dataprep/python_span_l_0b0ba655-6f7c-4edf-9a6a-ccb462e385ae.jsonl
outputs/titanic_model.pkl


In [20]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/titanic_model.pkl', model_name='Titanic',
                   tags={'Training context':'Estimator'},
                   properties={'Regularization Rate': run.get_metrics()['Regularization Rate'],
                       'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Model(workspace=Workspace.create(name='agogemls', subscription_id='da21a094-26a3-472f-991b-e2b11979af40', resource_group='agoge'), name=Titanic, id=Titanic:3, version=3, tags={'Training context': 'Estimator'}, properties={'Regularization Rate': '0.4', 'AUC': '0.822945283882784', 'Accuracy': '0.7947761194029851'})